In [0]:
%run validations2/fnSetupMountPoints

'year=2024/Month=02/day=27'

In [0]:
#mount_stage() creating mount points for the Testing adlv vs datalake
adlsContainerName = 'xyenta-dev-96-synapse'
storageAccountName = 'adlsxyentadevuks96'

adlsFolderName = 'Test'

mountPoint = "/mnt/DeltaLake/Test"

mount_stage(adls_container_name=adlsContainerName, storage_account_name= storageAccountName, adls_folder_name= adlsFolderName, mount_point= mountPoint)

abfss://xyenta-dev-96-synapse@adlsxyentadevuks96.dfs.core.windows.net/Test/


'year=2024/Month=02/day=26'

In [0]:
#dbutils.fs.unmount('/mnt/DeltaLake/Tes')

/mnt/DeltaLake/Tes has been unmounted.


True

In [0]:
dbutils.fs.ls('/mnt/DeltaLake')

[FileInfo(path='dbfs:/mnt/DeltaLake/Test/', name='Test/', size=0, modificationTime=0)]

In [0]:
#get the file path from mnt point DeltaLake
dbutils.fs.ls('/mnt/DeltaLake/Test')

path='dbfs:/mnt/DeltaLake/Test/SchemaManagementDelta.csv'

In [0]:
df = spark.read.format('csv').option('Header', True).option('inferSchema', True).load(path=path)

abfss://xyenta-dev-96-stg@adlsxyentadevuks96.dfs.core.windows.net/Orders/


abfss://xyenta-dev-96-stg@adlsxyentadevuks96.dfs.core.windows.net/People/


abfss://xyenta-dev-96-stg@adlsxyentadevuks96.dfs.core.windows.net/Returns/


[FileInfo(path='dbfs:/mnt/Dev96Stage/Orders/', name='Orders/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/Dev96Stage/People/', name='People/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/Dev96Stage/Returns/', name='Returns/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/Dev96Stage/year=2024/', name='year=2024/', size=0, modificationTime=1704510999000)]

In [0]:
df.show(10)
df.printSchema()

+-------------------+-----------+--------+----------+-------------+------+-------------+----------+
|    Education_Level|Line_Number|Employed|Unemployed|     Industry|Gender|Date_Inserted|dense_rank|
+-------------------+-----------+--------+----------+-------------+------+-------------+----------+
|  Bachelor's Degree|        326|  118000|      2000|  Agriculture|  Male|     6/1/1999|         1|
|High School Diploma|        195|11134000|    240000|  Agriculture|Female|     9/1/1996|         2|
|High School Diploma|        129|  675000|     15000|       Retail|Female|     5/1/1995|         3|
|    Master's Degree|        406|  131500|      3500|  Agriculture|  Male|     2/1/2001|         4|
|        High School|       1463| 4031000|    115000| Retail Trade|  Male|    12/1/2000|         5|
|  Bachelor's Degree|        426|  136900|      4100|Manufacturing|  Male|     7/1/2001|         6|
|  Bachelor's Degree|        458|  136900|      4100|  Agriculture|  Male|     3/1/2002|         6|


In [0]:
## write to parquet fromat 
op = f'dbfs:/mnt/DeltaLake/Test'

df.write.format('parquet').mode('overwrite').save(f'{op}/ParquetFolder/')

In [0]:
## read the parquet file 

df_parquet = spark.read.format('parquet').load(f'{op}/ParquetFolder/')

In [0]:
df_parquet.show()

+-------------------+-----------+--------+----------+-------------+------+-------------+----------+
|    Education_Level|Line_Number|Employed|Unemployed|     Industry|Gender|Date_Inserted|dense_rank|
+-------------------+-----------+--------+----------+-------------+------+-------------+----------+
|  Bachelor's Degree|        326|  118000|      2000|  Agriculture|  Male|     6/1/1999|         1|
|High School Diploma|        195|11134000|    240000|  Agriculture|Female|     9/1/1996|         2|
|High School Diploma|        129|  675000|     15000|       Retail|Female|     5/1/1995|         3|
|    Master's Degree|        406|  131500|      3500|  Agriculture|  Male|     2/1/2001|         4|
|        High School|       1463| 4031000|    115000| Retail Trade|  Male|    12/1/2000|         5|
|  Bachelor's Degree|        426|  136900|      4100|Manufacturing|  Male|     7/1/2001|         6|
|  Bachelor's Degree|        458|  136900|      4100|  Agriculture|  Male|     3/1/2002|         6|


In [0]:
## create tmep view 

df_parquet.createOrReplaceTempView('parquetView')

56

In [0]:
%sql
-- we will get error the reason error is , we are using Data Lake .Not Delta Lake. So in Data Lake updates the data/src file are not possible / supported 
update parquetView
set Education_Level = 'School'
where Education_Level = 'High School'

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.SparkUnsupportedOperationException: UPDATE TABLE is not supported temporarily.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.ddlUnsupportedTemporarilyError(QueryExecutionErrors.scala:1293)
	at org.apache.spark.sql.execution.SparkStrategies$BasicOperators$.apply(SparkStrategies.scala:1174)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$2(QueryPlanner.scala:69)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:69)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.plan(QueryPlanner.scala:100)
	at org.apache.spark.sql.execution.SparkStrategies.plan(SparkStrategies.scala:94)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$4(QueryPlanner.scala:85)
	at scala.collection.TraversableOnce$folder$1.apply(TraversableOnce.scala:196)
	at scala.collection.TraversableOnce$folder$1.apply(TraversableOnce.scala:194)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.TraversableOnce.foldLeft(TraversableOnce.scala:199)
	at scala.collection.TraversableOnce.foldLeft$(TraversableOnce.scala:192)
	at scala.collection.AbstractIterator.foldLeft(Iterator.scala:1431)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$3(QueryPlanner.scala:82)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.plan(QueryPlanner.scala:100)
	at org.apache.spark.sql.execution.SparkStrategies.plan(SparkStrategies.scala:94)
	at org.apache.spark.sql.execution.QueryExecution$.createSparkPlan(QueryExecution.scala:898)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$sparkPlan$1(QueryExecution.scala:365)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:400)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$4(QueryExecution.scala:431)
	at org.apache.spark.sql.execution.QueryExecution$.withInternalError(QueryExecution.scala:945)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$2(QueryExecution.scala:431)
	at com.databricks.util.LexicalThreadLocal$Handle.runWith(LexicalThreadLocal.scala:63)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:427)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:1138)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:427)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhaseWithTracker$1(QueryExecution.scala:442)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:166)
	at org.apache.spark.sql.execution.QueryExecution.executePhaseWithTracker(QueryExecution.scala:442)
	at org.apache.spark.sql.execution.QueryExecution.sparkPlan$lzycompute(QueryExecution.scala:365)
	at org.apache.spark.sql.execution.QueryExecution.sparkPlan(QueryExecution.scala:358)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executedPlan$1(QueryExecution.scala:379)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:1138)
	at org.apache.spark.sql.execution.QueryExecution.executedPlan$lzycompute(QueryExecution.scala:379)
	at org.apache.spark.sql.execution.QueryExecution.executedPlan(QueryExecution.scala:374)
	at org.apache.spark.sql.execution.QueryExecution.simple

In [0]:
## lets see with the Spark.sql (here also we will get error)  UPDATE TABLE is not supported temporarily.

spark.sql(""" update  parquetView set Education_Level = 'School'
                where Education_Level = 'High School' """)

---------------------------------------------------------------------------
UnsupportedOperationException             Traceback (most recent call last)
File <command-482608276354716>, line 3
      1 ## lets see with the Spark.sql (here also we will get error)
----> 3 spark.sql(""" update  parquetView set Education_Level = 'School'
      4                 where Education_Level = 'High School' """)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1602, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1598         assert self._jvm is not None
   1599         litArgs = self._jvm.PythonUtils.toArray(
   1600      

In [0]:
# filter and overwrite to another Temp directory , later on overwrite the parquet_folder with Temp

df_parquet = df_parquet.filter(col('Education_Level')== 'High School')

In [0]:
# get the Unique id of df. Even though Spark dfs are immutable, but we can modify the df's, beacuse python is a dynamically typed language and uses unique id of our df 
df_parquet.rdd.id() 

56

In [0]:

df_parquet.write.format('parquet').mode('overwrite').save(f"{op}/Temp")

In [0]:
#read from Temp and Overwrite the parquet_folder data . To understand the Versioning concept means overwrite is felt like unwanted and i just get back my previous unfiltered records. Here in Data Lake is not possible, we don't have versioning or Time Travel in Data Lake

df_temp = spark.read.format('parquet').load(f"{op}/Temp")

In [0]:
df_temp.write.format('parquet').mode('overwrite').save(f'{op}/ParquetFolder/')

##Creating DELTA LAKE

In [0]:
df.write.format('delta').mode('overwrite').save(f'{op}/Delta/')

In [0]:
#read Delta 

df_delta = spark.read.format('delta').load(f'{op}/Delta/')

In [0]:
df_delta.show()

+-------------------+-----------+--------+----------+-------------+------+-------------+----------+
|    Education_Level|Line_Number|Employed|Unemployed|     Industry|Gender|Date_Inserted|dense_rank|
+-------------------+-----------+--------+----------+-------------+------+-------------+----------+
|  Bachelor's Degree|        326|  118000|      2000|  Agriculture|  Male|     6/1/1999|         1|
|High School Diploma|        195|11134000|    240000|  Agriculture|Female|     9/1/1996|         2|
|High School Diploma|        129|  675000|     15000|       Retail|Female|     5/1/1995|         3|
|    Master's Degree|        406|  131500|      3500|  Agriculture|  Male|     2/1/2001|         4|
|        High School|       1463| 4031000|    115000| Retail Trade|  Male|    12/1/2000|         5|
|  Bachelor's Degree|        426|  136900|      4100|Manufacturing|  Male|     7/1/2001|         6|
|  Bachelor's Degree|        458|  136900|      4100|  Agriculture|  Male|     3/1/2002|         6|


In [0]:
#Now filter the data and overwrite to Same delta location for experiecning Data Versioning

df_delta_filter = df.filter(col('Education_Level')== 'High School')

df_delta_filter.write.format('delta').mode('overwrite').save(f'{op}/Delta/')

In [0]:
#Now read the filter records of delta files 

df_delta2 = spark.read.format('delta').load(f'{op}/Delta/').show()

+---------------+-----------+--------+----------+-------------+------+-------------+----------+
|Education_Level|Line_Number|Employed|Unemployed|     Industry|Gender|Date_Inserted|dense_rank|
+---------------+-----------+--------+----------+-------------+------+-------------+----------+
|    High School|       1463| 4031000|    115000| Retail Trade|  Male|    12/1/2000|         5|
|    High School|       1462| 4018000|    123000| Retail Trade|Female|    11/1/2000|         7|
|    High School|       1461| 4004000|    132000| Retail Trade|  Male|    10/1/2000|         9|
|    High School|       1460| 3990000|    140000| Retail Trade|Female|     9/1/2000|        10|
|    High School|       1459| 3974000|    149000| Retail Trade|  Male|     8/1/2000|        11|
|    High School|       1458| 3958000|    157000| Retail Trade|Female|     7/1/2000|        13|
|    High School|       1457| 3942000|    165000| Retail Trade|  Male|     6/1/2000|        17|
|    High School|       1456| 3925000|  